In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import load_img ,ImageDataGenerator,img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D ,MaxPool2D,Flatten

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_dir =r"D:\Data Science notes\AI\archive (3)\Covid19-dataset\train"
test_dir = r"D:\Data Science notes\AI\archive (3)\Covid19-dataset\test"

In [ ]:
width = 500
height = 500

batch=32

In [ ]:
train_dataset = ImageDataGenerator(
    rescale =1/255.0,
    rotation_range =20,
    height_shift_range =0.2,
    width_shift_range =0.2,
    shear_range =0.2,
    zoom_range =0.2,
    horizontal_flip =True
                    
)

In [ ]:
train_img =train_dataset.flow_from_directory(
    train_dir,
    target_size=(width,height),
    batch_size=batch,
    class_mode="categorical"
)

In [ ]:
test_dataset =ImageDataGenerator(rescale=1/255.0)

In [ ]:
test_img =test_dataset.flow_from_directory(
    test_dir,
    target_size=(width,height),
    batch_size=batch,
    class_mode="categorical"
)

In [ ]:
train_img.class_indices

In [ ]:
classes = {val:key for key,val in train_img.class_indices.items()}

In [ ]:
classes

In [ ]:
model =Sequential([
    Conv2D(64,(3,3),strides=(1,1),padding="valid",input_shape=(width,height,3),activation="relu"),
    MaxPool2D((2,2)),
    Conv2D(128,(3,3),strides=(1,1),padding="valid",activation="relu"),
    MaxPool2D((2,2)),
    Conv2D(256,(3,3),strides=(1,1),padding="valid",activation="relu"),
    MaxPool2D((2,2)),
    Flatten(),
    Dense(128,activation="relu"),
    Dense(256,activation="relu"),
    Dense(128,activation="relu"),
    Dense(3,activation="softmax"),
    
])

In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
history =model.fit(train_img,epochs=25,batch_size=batch,validation_data=test_img)

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(history.history["loss"],label="Training Loss")
plt.plot(history.history["val_loss"],label="Validation Loss")
plt.title("Loss visualization")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(history.history["accuracy"],label="Training Accuracy")
plt.plot(history.history["val_accuracy"],label="Validation Accuracy")
plt.title("Accuracy visualization")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
def image_predictor(img_path):
    img = load_img(img_path ,target_size=(32,32))
    img_arr = img_to_array(img)
    img_norm = img_arr /255.0
    img_flatten = np.expand_dims(img_norm ,axis=0)
    pred = model.predict(img_flatten)
    predicted_class = classes[np.argmax(pred[0])]
    return predicted_class